In [6]:
import pandas as pd
from stackapi import StackAPI
from pprint import pprint
from bs4 import BeautifulSoup
import time
import requests
import html5lib
import bs4
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed

In [7]:
df1 = pd.read_csv('QueryResults apt.csv')
df2 = pd.read_csv('QueryResults bottom 50.csv')
df3 = pd.read_csv('QueryResults command-line.csv')
df4 = pd.read_csv('QueryResults Scripts.csv')
df5 = pd.read_csv('QueryResults top 50.csv')
df6 = pd.read_csv('QueryResults bash.csv')

In [8]:
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [9]:
len(df)

72351

In [10]:
df.drop_duplicates(subset='Id', keep='first', ignore_index=True, inplace=True)

In [11]:
len(df)

57464

In [12]:
bash_file_codes=[]
failed=[]

In [13]:
for i in tqdm(range(len(df[0:4000]))):
    try:
        answers={}
        codes=[]
        i_d = df.iloc[i, 7]
        i_d = int(i_d)
        url = 'https://askubuntu.com/questions/'+str(i_d)
        r = requests.get(url) 
        soup = BeautifulSoup(r.content, 'html.parser')
        j=0

        q = soup.find('div', attrs={'class':"s-prose js-post-body", 'itemprop':"text"})



        for a in soup.findAll('div', attrs={'class':'answer accepted-answer'}):
            atext = a.text
            aid = a['id']
            ans = {}
            codes=[]
            c = a.find('div', attrs={'class':"js-vote-count grid--cell fc-black-500 fs-title grid fd-column ai-center", 'itemprop':"upvoteCount"})

            for b in a.findAll('div', attrs={'class':'s-prose js-post-body'}):
                for code in b.findAll('code'):
                    codes.append(code.text)

            ans['Answer Id'] = aid
            ans['Answer Text'] = atext

            if(c.text):
                ans['Votes'] = c.text
            else:
                ans['Votes'] = 0

            ans['Codes'] = codes

            answers['Accepted Answer'] = ans



        for a in soup.findAll('div', attrs={'class':'answer'}):
            atext = a.text
            aid = a['id']
            ans = {}
            codes=[]
            j=j+1
            c = a.find('div', attrs={'class':"js-vote-count grid--cell fc-black-500 fs-title grid fd-column ai-center", 'itemprop':"upvoteCount"})

            for b in a.findAll('div', attrs={'class':'s-prose js-post-body'}):
                for code in b.findAll('code'):
                    codes.append(code.text)

            ans['Answer Id'] = aid
            ans['Answer Text'] = atext

            if(c.text):
                ans['Votes'] = c.text
            else:
                ans['Votes'] = 0

            ans['Codes'] = codes
            answers['Answer'+str(j)] = ans

            if(j>=2):
                break

        bash_file_codes.append({'Id':i_d, 'Question Body': q.text, 'Answers':answers})
        pickle.dump(bash_file_codes, open('posts.txt', 'wb'))
        
    except:
        failed.append(i)
        pickle.dump(failed, open('failed.txt', 'wb'))
        




100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [2:07:32<00:00,  1.91s/it]
